### PROJECT-4. Решение комплексной бизнес-задачи


##### ЦЕЛЬ ПРОЕКТА: 
Подготовить основу рекомендательной системы курсов для онлайн-школы MasterMind.

##### ЗАДАЧИ:
1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями.

1. Создаем SQL-запрос для получения необходимых данных:

In [14]:
#WITH count_users AS (
  #SELECT 
    #user_id, 
    #COUNT (resource_id) AS count_cours 
  #FROM 
    #final.carts AS c 
    #JOIN final.cart_items AS ci ON ci.cart_id = c.id 
  #WHERE 
    #state = 'successful' 
    #AND resource_type = 'Course' 
  #GROUP BY 
    #user_id 
  #HAVING 
    #COUNT (DISTINCT resource_id) > 1
#) 
#SELECT 
  #DISTINCT count_users.user_id, 
  #resource_id 
#FROM 
  #final.carts AS c 
  #JOIN final.cart_items AS ci ON ci.cart_id = c.id 
  #RIGHT JOIN count_users ON c.user_id = count_users.user_id 
#WHERE 
  #state = 'successful' 
  #AND resource_type = 'Course' 
#ORDER BY 
  #count_users.user_id, 
  #resource_id

2. Обрабатываем данные средствами python:

In [15]:
#Загружаем библиотеки и данные, которые мы получили ранее с помощью sql-запроса
import pandas as pd
import numpy as np
import itertools as it
import collections as cl
sales_df = pd.read_csv('users.csv')

In [16]:
#Группируем покупки курсов по пользователям
courses_df = sales_df.groupby(['user_id'])['resource_id'].apply(lambda x:list(np.unique(x))).reset_index()
courses_df

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [17]:
#Разбиваем все покупки курсов по парам
courses = list(courses_df['resource_id'].apply(lambda y: list(it.combinations(y, 2))))
from itertools import chain
courses_list = list(chain.from_iterable(courses))

In [18]:
#Считаем, сколько раз встречается каждая пара
from collections import Counter
courses_pairs = Counter(courses_list)
courses_dict = dict(courses_pairs)

In [19]:
#Создаем функцию рекомендаций, которая на вход получает id курса и возвращает два самых часто продаваемых курса с тем, что получен на вход
def recomendations(course_id):
    recomm_list = []
    for i in courses_dict.keys():
        if i[0] == course_id:
            recomm_list.append((i, courses_dict[i]))
        elif i[1] == course_id:
            recomm_list.append((i, courses_dict[i]))
    recomm_list = sorted(recomm_list, key = lambda a: a[1], reverse=True)
    return recomm_list[:2]

In [20]:
#Создаем список с уникальными id курсов
сourse_list = np.unique(sales_df['resource_id']).tolist()

In [21]:
#Определяем самую популярную пару курсов
courses_pairs.most_common(1)

[((551, 566), 797)]

3. Создаем итоговую таблицу рекомендаций:

На основании подсчета количества встречаемости пар курсов, выбрали минимальную границу - 10


Для остальных курсов будем рекомендовать самые популярные курсы (551, 566)

In [22]:
rec_list = []
recommendations_df = pd.DataFrame(rec_list, columns=['first_rec', 'second_rec'])

min_value = 10

for course in сourse_list:
    if recomendations(course)[0][1]>= min_value:
        rec_1 = (set(recomendations(course)[0][0]) - set([course])).pop()
        if recomendations(course)[1][1] >= min_value:
            rec_2 = (set(recomendations(course)[1][0])-set([course])).pop()
        else:
            rec_2 = 551
        recommendations_df.loc[course] = [rec_1, rec_2]
    elif recomendations(course)[0][1] < min_value:
        recommendations_df.loc[course] = [551, 566]
        
recommendations_df.head(10)

,first_rec,second_rec
356,571,357
357,571,356
358,570,752
359,570,358
360,745,516
361,551,1138
362,363,551
363,511,562
364,551,566
365,551,566
